In [1]:
from googleapiclient.discovery import build
import yaml

import warnings
warnings.filterwarnings("ignore")

# Парсинг информации

В данной работе ставится задача тематического моделирования комментариев с YouTube, собранных с музыкальных обучающих и развлекательных каналов о барабанах. В этом ноутбуке написан парсер комментариев, который собирает и сохраняет данные в текстовый документ для дальнейшей обработки.

В папке data/source находится файл channels_id, в котором записаны 10 id номеров с каналов, которые буду парсить.

In [2]:
def comment_report(channel_id: str, apikey: str, nrequests: int) -> list[str]:
    """
    Возвращает список с комментариями с определенного канала
    :param channel_id: айди канала, с которого нужно взять комментарии
    :param apikey: ключ для получения доступа к API
    :param nrequests: количество запросов при выполнении функции (квота в 10000)
    :return: список с комментариями
    """
    service = build('youtube', 'v3', developerKey=apikey)

    args = {
        'allThreadsRelatedToChannelId': channel_id,
        'part': 'id,snippet,replies',
        'maxResults': 100
    }

    comments = []
    for page in range(nrequests):
        r = service.commentThreads().list(**args).execute()
        for top_level in r['items']:
            comment_id = top_level['snippet']['topLevelComment']['id']
            snippet = top_level['snippet']['topLevelComment']['snippet']
            comments.append(snippet['textOriginal'])

            if 'replies' in top_level:
                for reply in top_level['replies']['comments']:
                    comments.append(snippet['textOriginal'])

        args['pageToken'] = r.get('nextPageToken')
        if not args['pageToken']:
            break

    return comments

In [3]:
def summary_comments(channels_id: list[str], apikey: str, 
                     nrequests: int) -> list[str]:
    """
    Конкатенирует и возвращает список комментариев из разных запросов
    :param channels_id: список айди каналов
    :param apikey: ключ для получения доступа к API
    :param nrequests: количество запросов при выполнении функции (квота в 10000)
    :return: список с комментариями 
    """
    comments = []
    
    for idx in channels_id:
        comments += comment_report(idx, apikey, nrequests)

    return comments

In [4]:
def txt_file_to_list(file: str) -> list:
    """
    Преобразует информацию из файла в список, деля ее построчно
    :param file: путь до файла
    """
    with open(file, 'r', encoding='utf-8') as f:
        lst = f.readlines()
        lst = [i.strip() for i in lst]

    return lst

In [5]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

In [6]:
channels_id = config['parse']['channels_id']

In [7]:
ids = txt_file_to_list(channels_id)

In [8]:
apiKey = config['parse']['apiKey']
nrequests = config['parse']['nrequests']

In [10]:
comments = summary_comments(ids, apiKey, nrequests)

In [ ]:
def record_file(file: str, lst: list[str]) -> None:
    """
    Записывает в файл информацию из списка
    :param file: путь до файла
    :param lst: список строк
    """
    with open(file, 'w', encoding='utf-8') as f:
        for comment in lst:
            comment = comment.replace('\n', ' ')
            f.write(comment + '\n')

In [ ]:
comments_file = config['parse']['comments']

In [ ]:
record_file(comments_file, comments)